In [32]:
import pandas as pd
import os

In [33]:
# Cargar el archivo de ExcelH:\AGOSTO
archivo_entrada = 'F:\\Nominas\\Marzo\\Indice Marzo OK.xlsx'

In [34]:
# Ruta de las carpetas que deseas explorar
ruta_carpeta = 'F:\\Nominas\\Marzo\\Quincena_2305'
ruta_carpeta_2 = 'F:\\Nominas\\Marzo\\Quincena_2306'

In [35]:
# Leer el archivo de Excel y crear un DataFrame
df = pd.read_excel(archivo_entrada)

# Obtener la lista de nombres de archivos en la carpeta
nombres_archivos = os.listdir(ruta_carpeta)
# Obtener la lista de nombres de archivos en la segunda carpeta
nombres_archivos_2 = os.listdir(ruta_carpeta_2)

# Ruta de la segunda carpeta que deseas explorar
# ruta_carpeta_2 = '/ruta/a/la/segunda/carpeta'

# Seleccionar solo las columnas que deseas para trabajar
columnas_seleccionadas = ['ANOMINA', 'REPORTE', 'NARCHIVO', 'ARCHIVO', 'DEPENDEN', 'SUB_DEP', 'VALIDACIÓN']
df = df[columnas_seleccionadas]

# Eliminar las columnas que no necesitas
columnas_a_eliminar = ['ANOMINA', 'NARCHIVO', 'ARCHIVO', 'VALIDACIÓN']
df = df.drop(columns=columnas_a_eliminar)

# Crear un DataFrame con los nombres de los archivos
df_archivos = pd.DataFrame({'Nombre de Archivo': nombres_archivos})

# Crear un DataFrame con los nombres de los archivos de la segunda carpeta
df_archivos_2 = pd.DataFrame({'Nombre de Archivo': nombres_archivos_2})

# Verificar si el valor en la columna "SUB_DEP" es un número y reemplazarlo por un valor nulo si no lo es
df['SUB_DEP'] = df['SUB_DEP'].apply(lambda x: str(x) if (isinstance(x, (int, float)) and not pd.isna(x)) else " ")

# Definir una función para generar la nomenclatura
def generar_nomenclatura(row):
    reporte = str(row['REPORTE'])  # Convertir el valor a cadena por si acaso
    dependen = str(row['DEPENDEN'])  # Convertir el valor a cadena por si acaso
    sub_dep = str(row['SUB_DEP']).split('.')[0]  # Convertir el valor a cadena y eliminar contenido desde el punto decimal
    primeros_tres = reporte[:3]
    primeros_tres = reporte[:3]
    ultimos_cuatro = reporte[-4:]
    cuarto_digito = reporte[3]
    guion_bajo = f"_{sub_dep}" if sub_dep.isdigit() else ''
    return f"{primeros_tres}_{ultimos_cuatro}_{cuarto_digito}_{dependen}{guion_bajo}"

# Aplicar la función para generar la columna de nomenclatura
df['Nomenclatura'] = df.apply(generar_nomenclatura, axis=1)

# Mostrar la información requerida junto con la nueva columna de nomenclatura
#print(df[['REPORTE', 'DEPENDEN', 'SUB_DEP', 'Nomenclatura']])
print(df.head()[['Nomenclatura']])

# Crear una nueva columna en el DataFrame con los nombres de las nomenclaturas
# df['Nomenclatura'] = df.apply(lambda row: f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_{row['SUB_DEP']}" if isinstance(row['SUB_DEP'], int) else f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}" if pd.notnull(row['SUB_DEP']) else f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_", axis=1)

# df['Nomenclatura'] = df.apply(lambda row: f"{row['REPORTE'][:3]}_{row['REPORTE'][-4:]}_{row['REPORTE'][3]}_{row['DEPENDEN']}_{'' if pd.isna(row['SUB_DEP']) or not str(row['SUB_DEP']).isdigit() else '_' + str(row['SUB_DEP'])}", axis=1)

# Quitar la extensión de los nombres de archivo en el DataFrame
df_archivos['Nombre de Archivo'] = df_archivos['Nombre de Archivo'].apply(lambda x: os.path.splitext(x)[0])

# Quitar la extensión de los nombres de archivo de la segunda carpeta
df_archivos_2['Nombre de Archivo'] = df_archivos_2['Nombre de Archivo'].apply(lambda x: os.path.splitext(x)[0])

# Concatenar los DataFrames de ambas carpetas
df_final = pd.concat([df_archivos, df_archivos_2], ignore_index=True)

# Renombrar la columna
df_final = df_final.rename(columns={'Nombre de Archivo': 'Nomenclatura'})

# Mostrar los primeros registros del nuevo DataFrame
print(df_final.head())
#print(df_final.tail(50))

# Mostrar los primeros registros del nuevo DataFrame
# print(df_archivos.head())

# Obtener el número de archivos en cada DataFrame
num_archivos_df_1 = df_archivos.shape[0]
num_archivos_df_2 = df_archivos_2.shape[0]
num_archivos_df_final = df_final.shape[0]
df_total = df.shape[0]

# Mostrar el número de archivos en cada DataFrame
print("Número de archivos en el primer DataFrame:", num_archivos_df_1)
print("Número de archivos en el segundo DataFrame:", num_archivos_df_2)
print("Número de archivos en el DataFrame final:", num_archivos_df_final)
print("Número de archivos DF total:", df_total)

     Nomenclatura
0  NJA_2305_0_101
1  NJA_2305_0_102
2  NJA_2305_0_103
3  NJA_2305_0_104
4  NJA_2305_0_105
     Nomenclatura
0  NOA_2305_0_212
1  NOA_2305_3_212
2  NOA_2305_5_212
3  NOD_2305_0_212
4  NOD_2305_2_212
Número de archivos en el primer DataFrame: 552
Número de archivos en el segundo DataFrame: 704
Número de archivos en el DataFrame final: 1256
Número de archivos DF total: 1264


In [36]:
# Generar conjuntos de nomenclaturas para cada DataFrame
nomenclaturas_df = set(df['Nomenclatura'])
nomenclaturas_df_final = set(df_final['Nomenclatura'])

# Encontrar nomenclaturas que faltan en df_final
faltan_en_df_final = nomenclaturas_df - nomenclaturas_df_final

# Encontrar nomenclaturas que faltan en df
faltan_en_df = nomenclaturas_df_final - nomenclaturas_df

# Mostrar el número de archivos que faltan en cada DataFrame
print("Número de archivos que faltan en el DataFrame final:", len(faltan_en_df_final))
print("Número de archivos que faltan en el DataFrame original:", len(faltan_en_df))

# Mostrar la lista de nomenclaturas que no hacen match
print("Nomenclaturas que faltan en el DataFrame final:")
for nomenclatura in faltan_en_df_final:
    print(nomenclatura)

print("\nNomenclaturas que faltan en el DataFrame original:")
for nomenclatura in faltan_en_df:
    print(nomenclatura)

Número de archivos que faltan en el DataFrame final: 25
Número de archivos que faltan en el DataFrame original: 13
Nomenclaturas que faltan en el DataFrame final:
NOA_2306_B_222_1
NOD_2305_0_601
NOD_2305_0_504
NOD_2305_0_606
NOD_2305_0_603_5
NOD_2305_0_602
NOD_2306_4_224_1
NOJ_2305_0_801_1
NOA_2305_8_656
NOD_2305_0_512
NOD_2305_0_661
NOD_2305_2_504
NOD_2305_0_611
NOA_2305_4_605
NOA_2305_7_605
NOD_2305_0_512_1
NOD_2305_0_604
NOA_2305_1_656
NOA_2306_0_604
NOD_2305_0_662
NOD_2305_0_610
NOD_2305_0_603
NOA_2306_3_603_6
NJA_2305_0_211
NOJ_2305_0_801_2

Nomenclaturas que faltan en el DataFrame original:
NOA_2305_6_212
NOA_2306_6_611
NOA_2305_8_302 (2)
NOA_2305_0_305 (2)
NOA_2306_8_242 (2)
NOA_2306_3_503_6
desktop
NOA_2306_4_604
NOA_2305_8_260 (2)
NOA_2306_B_222
NOA_2305_5_308 (2)
NOD_2305_0_224_2 (2)
NOA_2305_1_653


In [37]:
# Convertir las columnas de nomenclatura en conjuntos
nomenclaturas_df = set(df['Nomenclatura'])
nomenclaturas_df_final = set(df_final['Nomenclatura'])

# Encontrar nomenclaturas que faltan en df_final
faltan_en_df_final = nomenclaturas_df - nomenclaturas_df_final

# Mostrar el número de archivos que faltan en el DataFrame final
print("Número de archivos que faltan en el DataFrame final:", len(faltan_en_df_final))

# Mostrar nomenclaturas que faltan en el DataFrame final
print("Nomenclaturas que faltan en el DataFrame final:")
for nomenclatura in faltan_en_df_final:
    print(nomenclatura)

Número de archivos que faltan en el DataFrame final: 25
Nomenclaturas que faltan en el DataFrame final:
NOA_2306_B_222_1
NOD_2305_0_601
NOD_2305_0_504
NOD_2305_0_606
NOD_2305_0_603_5
NOD_2305_0_602
NOD_2306_4_224_1
NOJ_2305_0_801_1
NOA_2305_8_656
NOD_2305_0_512
NOD_2305_0_661
NOD_2305_2_504
NOD_2305_0_611
NOA_2305_4_605
NOA_2305_7_605
NOD_2305_0_512_1
NOD_2305_0_604
NOA_2305_1_656
NOA_2306_0_604
NOD_2305_0_662
NOD_2305_0_610
NOD_2305_0_603
NOA_2306_3_603_6
NJA_2305_0_211
NOJ_2305_0_801_2
